In [2]:
!pip install nltk
!pip install language-tool-python
!pip install pyspellchecker
!pip install install-jdk

import pandas as pd
import numpy as np
import nltk
import jdk
import os
# download java
jdk.install('17')
# !export JAVA_HOME=$HOME/.jdk/jdk-17.0.17+10
# !export PATH=$PATH:$JAVA_HOME/bin
# jdk_version = 'jdk-17.0.17+10' #change with your version 
#os.environ['JAVA_HOME'] = '/home/jovyan/.jdk/jdk-17.0.17+10'
#os.environ['PATH'] = f"{os.environ.get('PATH')}:{os.environ.get('JAVA_HOME')}/bin"
# for word_tokenize
nltk.download('punkt_tab')
nltk.download('punkt')
# for pos_tag
nltk.download('averaged_perceptron_tagger_eng')
# grammar error
import language_tool_python
tool = language_tool_python.LanguageTool('en-US')
# spell checker
from spellchecker import SpellChecker
spell = SpellChecker(language='en')
# parallelism for faster compute
from multiprocessing import Pool, cpu_count

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached regex-2025.11.3-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
Using cached regex-2025.11.3-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (800 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached pyspellchecker-0.8.3-py3-none-any.whl.metadata (9.5 kB)
Using cached pyspellchecker-0.8.3-py3-none-any.whl (7.2 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
  Using cached install_jdk-1.1.0-py3-none-any.whl.metadat

[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


ModuleNotFoundError: No java install detected. Please install java to use language-tool-python.

In [74]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [3]:
CONTENT_TAGS = {
    # Nouns
    "NN", "NNS", "NNP", "NNPS",
    # Verbs
    "VB", "VBD", "VBG", "VBN", "VBP", "VBZ",
    # Adjectives
    "JJ", "JJR", "JJS",
    # Adverbs
    "RB", "RBR", "RBS",
}

DERIVATIONAL_SUFFIXES = {
    "tion", "sion", "ment", "ness", "ity", "ance", "ism", "ship", "hood",
    "ive", "ous", "able", "al", "ical", "ize", "ise", "ify", "ate",
    "ary", "ory", "ant", "ent", "ery", "ist"
}

def compute(text: str, total_words: int) -> np.ndarray:
    text = str(text)
    total_words = int(total_words)

    # sentences and sentence lengths
    sentences = nltk.sent_tokenize(text)
    sent_lengths = [
        len(nltk.word_tokenize(sent))
        for sent in sentences
    ]

    tokens = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokens)

    # Feature 1
    num_sentences = len(sentences)

    # Feature 2
    avg_sent_len = total_words / num_sentences

    # Feature 3
    std_sent_len = float(np.std(sent_lengths))

    # Feature 4
    cleaned_tokens = [
        w.replace('.', '').replace(',', '')
        for w in tokens
    ]
    total_chars = sum(len(w) for w in cleaned_tokens)
    chars_per_word = total_chars / total_words

    # Feature 5
    content_words = sum(1 for _, tag in tagged if tag in CONTENT_TAGS)
    content_ratio_val = content_words / total_words

    # Feature 6
    total_suffixes = 0
    for w in tokens:
        wl = w.lower()
        if len(wl) > 5 and any(wl.endswith(suffix) for suffix in DERIVATIONAL_SUFFIXES):
            total_suffixes += 1
    deriv_ratio_val = total_suffixes / total_words

    # Feature 7
    matches = tool.check(text)
    error_ratio_val = (len(matches) / total_words)

    # Feature 8
    misspelled = spell.unknown(tokens)
    spelling_error_ratio = len(misspelled) / total_words
    
    return np.array([
        num_sentences,
        avg_sent_len,
        std_sent_len,
        chars_per_word,
        content_ratio_val,
        deriv_ratio_val,
        error_ratio_val,
    ], dtype=np.float32)

In [ ]:
df = pd.read_csv("english_exam_database.csv")

In [5]:
def compute_row(args):
    text, wc = args
    return compute(text, wc)

In [6]:
texts = df["text_corrected"].to_numpy()
wordcounts = df["wordcount"].to_numpy()

pairs = list(zip(texts, wordcounts))
cpu_num = cpu_count()

with Pool(processes=cpu_num) as pool:
    feature_rows = pool.map(compute_row, pairs, chunksize=100)

text_features = np.vstack(feature_rows)
print(text_features.shape)

NameError: name 'cpu_count' is not defined

In [7]:
features = df[["wordcount", "mtld"]].to_numpy(dtype=np.float32)
print(features.shape)

(317220, 2)


In [18]:
X = np.concatenate([features, X], axis=1)
print(X.shape)

(317220, 11)


In [57]:
# Should be N by 10
print(X.shape)

NameError: name 'X' is not defined

**Training the Model**

In [10]:
print(df[df['cefr'] == 'A1'].shape[0])
print(df[df['cefr'] == 'A2'].shape[0])
print(df[df['cefr'] == 'B1'].shape[0])
print(df[df['cefr'] == 'A2'].shape[0])
print(df[df['cefr'] == 'C1'].shape[0])

149492
85753
55033
85753
4891


In [4]:
y = np.array([1, 2, 3, 4, 5])

In [13]:
y = df[["cefr_numeric"]].to_numpy(dtype=np.float32)
y

array([[1.],
       [1.],
       [1.],
       ...,
       [5.],
       [5.],
       [5.]], dtype=float32)

array([[1.],
       [1.],
       [1.],
       ...,
       [5.],
       [5.],
       [5.]], dtype=float32)

In [15]:
def stratified_train_test_split(X: np.ndarray, y: np.ndarray):
    train_ratio = 0.8
    # turn y into 1 x N s.t. N is the number of data points
    y = y.ravel()
    train_ratio = 0.8

    unique_classes = np.unqiue(y) # 1,2,3,4,5

    train_indices = []
    test_indices = []

    for c in unique_classes:
        class_indices = np.where(y == c)[0]
        np.random.shuffle(class_indices)
        n_class = len(class_indices)
        n_train_c = int(train_ratio * n_class)

        train_indices.append(class_indices[:n_train_c])
        test_indices.append(class_indices[n_train_c:])

    # concatenate indices from all classes
    train_indices = np.concatenate(train_indices)
    test_indices = np.concatenate(test_indices)

    np.random.shuffle(train_indices)
    np.radom.shuffle(test_indices)

    X_train = X[train_indices]
    y_train = y[train_indices]
    X_test = X[test_indices]
    y_test = y[test_indices]

    return X_train, y_train, X_test, y_test

In [ ]:
# NOTE TO SELF: KEEP THESE TWO VARIABLES TO PREDICT NEW DATA LATER
feature_means = X_train.mean(axis=0) # shape (10,)
feature_stds = X_train.std(axis=0) # shape (10,)

# edge case to avoid division by zero
feature_std[feature_stds == 0] = 1.0

# standardize train and test using the training info
X_train_std = (X_train - feature_means) / feature_stds
X_test_std = (X_test - feature_means) / feature_stds

In [ ]:
# change index for softmax to be easier
y_train_classifier = y_train.astype(int) - 1
y_test_classifier = y_test.astype(int) - 1